# Defining and Detecting Interactions
## Distances between Bees

In [13]:
import os
import math
from collections import Counter

import numpy as np
import scipy as sc

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from bb_binary import FrameContainer, Repository

matplotlib.rc("figure", figsize=(12, 6))
sns.set_style('white')

repo = Repository('/Users/alexa/Projects/bees-sna/data-13hour/2015/')

TypeError: find() missing 1 required positional argument: 'ts'

In [14]:
for fname in repo.iter_fnames():
    print(fname)
#for fname in repo.iter_fnames():
   # print(fname)

ValueError: not enough values to unpack (expected 3, got 1)

In [ ]:
all_distances = [] # list of all distances (distance, bee1, bee2, frameID, angle)

for frame, container in repo.iter_frames():
    positions = []
    for detection in frame.detectionsUnion.detectionsDP:
        positions.append((detection.xpos, detection.ypos, detection.decodedId, detection.zRotation))

    # Calculate distances for this frame
    for i in range(len(positions)-2):
        bee1 = positions[i]
        for j in range(i+1, len(positions)-1):
            bee2 = positions[j]
            distX = bee2[0] - bee1[0]
            distY = bee2[1] - bee1[1]
            dist = math.sqrt(distX*distX + distY*distY)
            angle = math.degrees(abs(bee2[3] - bee1[3]))
            if angle > 180:
                angle = 360 - angle
            all_distances.append((dist, bee1, bee2, frame.frameIdx, angle))

Why is the ID (# global unique id of the frame) always 0?

Number of all distances over all frames

In [ ]:
print(len(all_distances))

Minimum and Maximal Distance between bees


In [ ]:
list = [x[0] for x in all_distances]
dists = np.array(list)
print("Minimum distance: {}".format(np.min(dists)))
print("Maximum distance: {}".format(np.max(dists)))
print("Mean distance: {}".format(np.mean(dists)))
print("Median distance: {}".format(np.median(dists)))
print("STD: {}".format(np.std(dists)))

## Histogramm of Distances

In [ ]:
hist, bins = np.histogram(dists, bins=100)
width = 0.5* (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

## What value to use for close distance?
How to choose the right distance? How to calculate it?

In [ ]:
max = 1000
list = []

for i in range(0, max, 10): 
    list.append(len([x for x in all_distances if x[0] <= i]))

In [ ]:
print(list)
plt.plot(list, 'bo', markersize=3)
plt.show()

In [ ]:
max = 100
l = [x for x in all_distances if x[0] <= max]
print("Number of distances: {}".format(len(l)))

## Calculating IDs for close bees
What IDs are involved? For each succesive timeframe, do the same bees stay together?
First calculate all the IDs, those who can not be translated for sure (63) are just thrown away.

In [ ]:
# Calculating ids with delta 63 (just for testing)
def getId(delta, id):

    c = 0
    number = 0
    for i in range(len(id)):
        if (id[i] <= 0 + delta):
            c = c + 1
        if (id[i] >= 255 - delta):
            c = c + 1
            number = number + 2**i
    # Add detected number in decimal
    if c == 12:
        return number
    else:
        return -1
    
delta = 63

id_list = [x for x in l if ((getId(delta, x[1][2]) > -1) and (getId(delta, x[2][2])) > -1)]    

In [ ]:
print(id_list[0])
print("Number of good ids: {}".format(len(id_list)))

In [ ]:
percent = ((100*len(id_list))/len(l))
print("Percent of bees with good ID: {}%".format(percent))

So we lost kind of a lot of bees when calculating the IDs in decimal.

In [ ]:
bees = [] #IDbee1, IDbee2, bee1, bee2, frameID, distance, angle
for x in id_list:
    bees.append((getId(delta, x[1][2]), getId(delta, x[2][2]), x[1], x[2], x[3], x[0], x[4]))
    

In [ ]:
print(bees[2])
#print(id_list[2])

# Angles - zRotation between pairs of Bees
## Look at the angles
* What angles? Range|min|max|mean|median
* Whats the distribution?
* How many bees over 70 deg?

In [ ]:
angles = [x[6] for x in bees]
a = np.array(angles)
print("Minimum angle: {}".format(np.min(a)))
print("Maximum angle: {}".format(np.max(a)))
print("Mean angle: {}".format(np.mean(a)))
print("Median angles: {}".format(np.median(a)))
print("STD: {}".format(np.std(a)))

In [ ]:
print(bees[0])

## Angle Distribution

In [ ]:
hist, bins = np.histogram(angles, bins=60)
width = 0.5* (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

## How many degrees over 70 degree?
Why 70? Why not 90?

I think trophallaxis can happen between more than two bees.

http://www.michiganbees.org/2012/trophallaxis/

# Frequency of Interaction
* Are there any ID combinations which occur very often?
* How often is often?

In [ ]:
from collections import Counter

d = []
for elem in bees:
    d.append(frozenset([elem[0], elem[1]]))

In [ ]:
print(len(d))
print(len(Counter(d)))
# print(Counter(d))

In [ ]:
countings = [value for key,value in Counter(d).items()]

c = np.array(countings)
print("Minimum countings: {}".format(np.min(c)))
print("Maximum countings: {}".format(np.max(c)))
print("Mean countings: {}".format(np.mean(c)))
print("Median countings: {}".format(np.median(c)))
print("STD: {}".format(np.std(c)))


In [ ]:
sc.stats.itemfreq(countings)

## Those bees who meet kind of often, where do they meet.
Plot for highest three pairs.

In [ ]:
pairs = []
for key, value in Counter(d).items():
    if(value > 30):
        print(key, value)
        pairs.append(key)

for p in pairs:
    interactions = [x for x in bees if(x[0] in p and x[1] in p)]
    bees1 = [(x[2][0],x[2][1]) for x in interactions]
    bees2 = [(x[3][0],x[3][1]) for x in interactions]
    
    # plotten
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.scatter(*zip(*bees1), s=10, c='b', marker="o", label='bee1')
    ax1.scatter(*zip(*bees2), s=10, c='r', marker="o", label='bee2')
    plt.legend(loc='upper right');
    plt.xlim(0, 3000)
    plt.ylim(0, 3000)
    plt.axes().set_aspect('equal')
    plt.suptitle('Bee-pair {}'.format(p), fontsize=12)
    plt.show()

## Whats the angle of those bee pairs?

In [ ]:
for p in pairs:
    interactions = [x[6] for x in bees if(x[0] in p and x[1] in p)]
    
    hist, bins = np.histogram(interactions, bins=60)
    width = 0.5* (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.bar(center, hist, align='center', width=width)
    plt.xlim(0, 180)
    plt.ylim(0, 20)
    plt.suptitle('Bee-pair with numbers {}'.format(p), fontsize=12)
    plt.show()

## Just make a network.

In [ ]:
import networkx as nx
G=nx.Graph()

In [ ]:
edges = []
for key, value in Counter(d).items():
    if (value >= 9):
        elem1, elem2 = key
        edges.append((elem1,elem2))

In [ ]:
print(edges[0])
print(len(edges))

In [ ]:
# without edge weights
print(len(set(edges)))

In [ ]:
G.add_edges_from(edges)
pos=nx.fruchterman_reingold_layout(G)

#pos = nx.layout.fruchterman_reingold_layout(g, k=0.85, pos=pos)
nx.draw(G,pos,node_color='#A0CBE2',edge_color='#BBBBBB',width=1,edge_cmap=plt.cm.Blues,with_labels=True)
plt.show()

In [ ]:
nx.write_graphml(G, "test.graphml")

In [ ]:
print(nx.number_of_nodes(G))

In [ ]:
nx.density(G)

In [ ]:
nx.info(G)

In [ ]:
degrees = nx.degree_histogram(G)
print(degrees)

plt.plot(degrees, 'bo', markersize=5)

plt.xlabel('Frame')
plt.ylabel('Detections')
plt.title('Detections per Frame')
plt.axis([0, 20, 0, 40])
plt.grid(True)

plt.show()

In [ ]:
nx.is_connected(G)
nx.number_connected_components(G)
components = sorted(nx.connected_components(G), key = len, reverse=True)
print([len(x) for x in components])

## Only look at the largest component

In [ ]:
components = sorted(nx.connected_component_subgraphs(G), key = len, reverse=True)
gc = components[0]

pos = nx.fruchterman_reingold_layout(gc)
nx.draw(gc,pos,node_color='#A0CBE2',edge_color='#BBBBBB',width=1,edge_cmap=plt.cm.Blues,with_labels=True)
plt.show()

In [ ]:
nx.write_graphml(gc, "gc.graphml")